In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Set seaborn style
sns.set(style="whitegrid", palette="colorblind")

# Constants
P_TO_KG_HA = 1.12085  # lb/acre to kg/ha
PLOT_AREA_HA = 0.002  # plot area in hectares

# Load data
#df = pd.read_csv('yield_phosfert_merged.csv')
df = pd.read_csv(r'C:\Users\frank\OneDrive - Mississippi State University\Research_0924\Phosphorus fertilizer\da_phosfert_0125\yield_phosfert_merged.csv')

# Summarize yield by phosphorus application
yield_summary = df.groupby('P_appln', as_index=False)['Mkt_wt'].mean()

# Save summary
yield_summary.to_csv('yield_avgs_021825.csv', index=False)

# Prepare data for plotting and modeling
melted_df = yield_summary.rename(columns={'Mkt_wt': 'Average Yield'})
melted_df['Root_quality'] = 'Marketable roots'
melted_df['Yield'] = melted_df['Average Yield'] / 1000 / PLOT_AREA_HA  # kg/plot to Mg/ha
melted_df['Phosphorus Fertilizer'] = melted_df['P_appln'] * P_TO_KG_HA  # lb/acre to kg/ha
# Quadratic regression and plotting
def quadratic_regression(df):
    X = df[['Phosphorus Fertilizer']]
    y = df['Yield']

    # Polynomial features
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)

    # Fit model
    model = LinearRegression().fit(X_poly, y)
    y_pred = model.predict(X_poly)

    # R² score
    r_squared = model.score(X_poly, y)

    # Optimal point
    best_idx = np.argmax(y_pred)
    best_phos = X.iloc[best_idx, 0]
    best_yield = y_pred[best_idx]

    # Coefficients
    a, b = model.coef_
    c = model.intercept_

    # Plot
    sorted_idx = X['Phosphorus Fertilizer'].argsort()
    plt.figure(figsize=(8, 6))
    plt.scatter(X, y, label='Observed', color=sns.color_palette()[0])
    plt.plot(X.iloc[sorted_idx], y_pred[sorted_idx], label='Quadratic Fit', color=sns.color_palette()[1])
    plt.scatter(best_phos, best_yield, color=sns.color_palette()[2], s=150, edgecolor='black', label='Optimum')

    equation = f'$y = {a:.2f}x^2 + {b:.2f}x + {c:.2f}$'
    plt.title(f'{equation}\n$R^2 = {r_squared:.2f}$', fontsize=16)
    plt.xlabel('Phosphorus Fertilizer (kg/ha)', fontsize=14)
    plt.ylabel('Yield (Mg/ha)', fontsize=14)
    plt.legend()
    sns.despine()
    plt.tight_layout()
    plt.show()

    return {
        'intercept': c,
        'coefficients': [a, b],
        'r_squared': r_squared,
        'best_phosphorus_amount': best_phos,
        'best_yield': best_yield
    }

# Run regression and display results
results = quadratic_regression(melted_df)
print(results)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Read the data
#df = pd.read_csv('merged_phosfert_021125.csv')
#df = pd.read_csv('yield_phosfert_merged.csv')
df = pd.read_csv(r'C:\Users\frank\OneDrive - Mississippi State University\Research_0924\Phosphorus fertilizer\da_phosfert_0125\yield_phosfert_merged.csv')

df.info()

# Group by Variety, Year, and P_appln and calculate mean yields
yield_summary = df.groupby(['P_appln']).agg({
    #'No.1_wt': 'mean',
    #'Can_wt': 'mean',
    #'Jumbo_wt': 'mean',
    'Mkt_wt': 'mean'
}).reset_index()

yield_summary.head()
yield_summary.info()

# Save the summary to CSV
yield_summary.to_csv('yield_avgs_021825.csv', index=False)

# Define a mapping to rename the variables for plotting
name_mapping = {
    #'No.1_wt': 'US_No.1',
    #'Can_wt': 'Canner',
    #'Jumbo_wt': 'Jumbo',
    'Mkt_wt': 'Marketable roots'
}

# Melt the DataFrame and apply renaming
melted_df = yield_summary.melt(id_vars=['P_appln'], 
                                value_vars=list(name_mapping.keys()),
                                var_name='Root_quality', value_name='Average Yield')

# Apply the renaming mapping to 'Root_quality'
melted_df['Root_quality'] = melted_df['Root_quality'].replace(name_mapping)

melted_df.head()

In [ ]:
# CONVERSIONS
# Yield from kg/plot to Mg/ha
import pandas as pd

# Plot area in hectares
plot_area_ha = 0.0009282

melted_df['Yield'] = melted_df['Average Yield'] / 1000 / plot_area_ha


# P_appln' from pounds per acre (lb/A) to kilograms per hectare (kg/ha)

melted_df['Phosphorus Fertilizer'] = melted_df['P_appln'] * 1.12085


In [ ]:
melted_df.info()
melted_df.head()


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Use seaborn's colorblind palette
colors = sns.color_palette("colorblind")

def quadratic_regression(melted_df):
    """
    Fits a quadratic regression model to phosphorus fertilizer vs. yield data,
    plots the results, and returns model statistics and optimal fertilizer amount.
    
    Parameters:
        melted_df (pd.DataFrame): DataFrame with 'Phosphorus Fertilizer' and 'Yield' columns.
    
    Returns:
        dict: Model coefficients, R², and optimal phosphorus application.
    """
    # Validate input
    required_columns = {'Phosphorus Fertilizer', 'Yield'}
    if not required_columns.issubset(melted_df.columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_columns}")
    
    X = melted_df[['Phosphorus Fertilizer']]
    y = melted_df['Yield']
    
    # Fit quadratic regression
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression().fit(X_poly, y)
    y_pred = model.predict(X_poly)
    
    # R-squared
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - y.mean()) ** 2)
    r_squared = 1 - ss_res / ss_tot
    
    # Optimal phosphorus application
    best_index = np.argmax(y_pred)
    best_phosphorus = X.iloc[best_index, 0]
    best_yield = y_pred[best_index]
    
    # Coefficients
    coef = model.coef_
    intercept = model.intercept_
    
    # Plotting
    plt.figure(figsize=(8, 6))
    sorted_idx = X['Phosphorus Fertilizer'].argsort()
    
    plt.scatter(X, y, color=colors[0])
    plt.plot(X.iloc[sorted_idx], y_pred[sorted_idx], color=colors[1], linewidth=2, label='Quadratic Fit')
    plt.scatter(best_phosphorus, best_yield, color=colors[2], s=150, edgecolor='black', zorder=5, label='Optimum')
    
    # Equation annotation
    a, b = coef
    c = intercept
    equation = f'$y = {a:.2f}x^2 + {b:.2f}x + {c:.2f}$'
    plt.title(f'{equation}\n$R^2 = {r_squared:.2f}$', fontsize=20)
    
    plt.xlabel('Phosphorus Fertilizer (kg/ha)', fontsize=22)
    plt.ylabel('Average Root Weight (Mg/ha)', fontsize=22)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=18)
    plt.grid(False, linestyle='', alpha=0.6)
    sns.despine()
    plt.tight_layout()
    plt.show()
    
    return {
        'intercept': c,
        'coefficients': coef.tolist(),
        'r_squared': r_squared,
        'best_phosphorus_amount': best_phosphorus,
        'best_yield': best_yield
    }



results = quadratic_regression(melted_df)
print(results)
